<a href="https://colab.research.google.com/github/8Raouf24/TP_RC/blob/main/Bayesian_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pgmpy

In [ ]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD

#Création du DAG
model = BayesianModel([('A', 'E'), ('A', 'F'), ('X', 'F'), ('F', 'C'), ('E', 'C'), ('C', 'S'), ('C', 'T')])

# Définition des cpds ( Conditionals Probability Distributions)
cpd_a = TabularCPD(variable='A', variable_card=2, values=[[0.7], [0.3]], state_names={'A': ['Enfant', 'Adulte']})

cpd_x = TabularCPD(variable='X', variable_card=2, values=[[0.4], [0.6]], state_names={'X': ['Homme', 'Femme']})

cpd_e = TabularCPD(variable='E', variable_card=2, values=[[0.95,0.7], [0.05,0.3]],
                   evidence=['A'],evidence_card=[2], state_names={'E': ['Oui', 'Non'],'A': ['Enfant', 'Adulte']})

cpd_f = TabularCPD(variable='F', variable_card=2, values=[[0.35,0.1,0.01,0], [0.65,0.9,0.99,1]],
                   evidence=['A', 'X'],evidence_card=[2, 2],state_names={'F': ['Fumeur', 'Non_Fumeur'],'A': ['Enfant', 'Adulte'],'X': ['Homme', 'Femme']})

cpd_c = TabularCPD(variable='C', variable_card=2, values=[[0.85,0.5,0.3,0.2], [0.15,0.5,0.7,0.8]],
                   evidence=['E', 'F'],evidence_card=[2, 2],state_names={'C': ['Cancereux', 'Sain'],'E': ['Oui', 'Non'],'F': ['Fumeur', 'Non_Fumeur']})

cpd_s = TabularCPD(variable='S', variable_card=2, values=[[0.2,0.03], [0.8,0.97]],evidence=['C'],
                   evidence_card=[2],state_names={'S': ['Elevé', 'Bas'],'C': ['Cancereux', 'Sain']})

cpd_t = TabularCPD(variable='T', variable_card=2, values=[[0.9,0.3], [0.1,0.7]],evidence=['C'],
                   evidence_card=[2],state_names={'T': ['Malade', 'Non_Malade'],'C': ['Cancereux', 'Sain']})

model.add_cpds(cpd_a, cpd_x, cpd_e, cpd_f, cpd_c, cpd_s,cpd_t)
model.check_model()

True

In [ ]:
model.get_cpds()

[<TabularCPD representing P(A:2) at 0x7fde56ead910>,
 <TabularCPD representing P(X:2) at 0x7fddfa9bcb90>,
 <TabularCPD representing P(E:2 | A:2) at 0x7fde56eadd50>,
 <TabularCPD representing P(F:2 | A:2, X:2) at 0x7fde56eadc10>,
 <TabularCPD representing P(C:2 | E:2, F:2) at 0x7fde56ead990>,
 <TabularCPD representing P(S:2 | C:2) at 0x7fde56ead550>,
 <TabularCPD representing P(T:2 | C:2) at 0x7fde56ead590>]

In [ ]:
print(cpd_e)

+--------+-----------+-----------+
| A      | A(Enfant) | A(Adulte) |
+--------+-----------+-----------+
| E(Oui) | 0.95      | 0.7       |
+--------+-----------+-----------+
| E(Non) | 0.05      | 0.3       |
+--------+-----------+-----------+


In [ ]:
print(cpd_f)

+---------------+-----------+-----------+-----------+-----------+
| A             | A(Enfant) | A(Enfant) | A(Adulte) | A(Adulte) |
+---------------+-----------+-----------+-----------+-----------+
| X             | X(Homme)  | X(Femme)  | X(Homme)  | X(Femme)  |
+---------------+-----------+-----------+-----------+-----------+
| F(Fumeur)     | 0.35      | 0.1       | 0.01      | 0.0       |
+---------------+-----------+-----------+-----------+-----------+
| F(Non_Fumeur) | 0.65      | 0.9       | 0.99      | 1.0       |
+---------------+-----------+-----------+-----------+-----------+


In [ ]:
print(cpd_c)

+--------------+-----------+---------------+-----------+---------------+
| E            | E(Oui)    | E(Oui)        | E(Non)    | E(Non)        |
+--------------+-----------+---------------+-----------+---------------+
| F            | F(Fumeur) | F(Non_Fumeur) | F(Fumeur) | F(Non_Fumeur) |
+--------------+-----------+---------------+-----------+---------------+
| C(Cancereux) | 0.85      | 0.5           | 0.3       | 0.2           |
+--------------+-----------+---------------+-----------+---------------+
| C(Sain)      | 0.15      | 0.5           | 0.7       | 0.8           |
+--------------+-----------+---------------+-----------+---------------+


In [ ]:
print(cpd_s)

+----------+--------------+---------+
| C        | C(Cancereux) | C(Sain) |
+----------+--------------+---------+
| S(Elevé) | 0.2          | 0.03    |
+----------+--------------+---------+
| S(Bas)   | 0.8          | 0.97    |
+----------+--------------+---------+


In [ ]:
print(cpd_t)

+---------------+--------------+---------+
| C             | C(Cancereux) | C(Sain) |
+---------------+--------------+---------+
| T(Malade)     | 0.9          | 0.3     |
+---------------+--------------+---------+
| T(Non_Malade) | 0.1          | 0.7     |
+---------------+--------------+---------+


In [ ]:
from pgmpy.inference import VariableElimination
infer = VariableElimination(model)
c_dist = infer.query(['C'],evidence={'A':'Adulte'})
print(c_dist)

Eliminating: S: 100%|██████████| 5/5 [00:00<00:00, 434.33it/s]

+--------------+----------+
| C            |   phi(C) |
+==============+==========+
| C(Cancereux) |   0.4111 |
+--------------+----------+
| C(Sain)      |   0.5889 |
+--------------+----------+
